In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

"Director: Jean-Pierre Jeunet"
"Main Cast: Audrey Tautou"
"Genre: Romantic comedy"
"Synopsis: A whimsical young woman in Paris decides to change the lives of those around her for the better while struggling with her own isolation and longing for love."

AIMessage(content='"Director: Jean-Pierre Jeunet"\n"Main Cast: Audrey Tautou"\n"Genre: Romantic comedy"\n"Synopsis: A whimsical young woman in Paris decides to change the lives of those around her for the better while struggling with her own isolation and longing for love."', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-3.5-turbo-0125', 'service_tier': 'default'}, id='run--5922ad20-9b24-4d98-baa3-07a179f5cb30-0')